In [1]:
#import all packages
import pandas as pd
import numpy as np
import re as re
import nltk
import string, unicodedata
from nltk.stem.porter import *

from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from datetime import datetime, timedelta

In [2]:
def reduce_lengthening(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)

In [3]:
# datafile
data_file = "data_with_topic_v2.csv"
data = pd.read_csv(data_file)
data.columns = data.columns.str.strip().str.replace(" ","_") # reason for doing this is cause cannot get columns with space into a list

#reformat the respective columns

#lower case
data['Review'] = [word.lower() for word in data['Review']]

#Strip off the Variation Word
data['Product_Purchase'] = data['Product_Purchase'].replace('Variation', '')

#Change Date of Review to DateTime
#data['Date_Of_Review'] = pd.to_datetime(data['Date_Of_Review'])

data.info()
data.head()
#data.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8987 entries, 0 to 8986
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed:_0        8987 non-null   int64  
 1   Platform          8987 non-null   object 
 2   Brand             8987 non-null   object 
 3   Category          8987 non-null   object 
 4   Product_Name      8987 non-null   object 
 5   Price             8987 non-null   float64
 6   Reviewer          8965 non-null   object 
 7   Review            8987 non-null   object 
 8   Product_Purchase  7382 non-null   object 
 9   Ratings           8965 non-null   float64
 10  Date_Of_Review    8965 non-null   object 
 11  Response          8965 non-null   object 
 12  topic             8987 non-null   object 
dtypes: float64(2), int64(1), object(10)
memory usage: 912.9+ KB


,Unnamed:_0,Platform,Brand,Category,Product_Name,Price,Reviewer,Review,Product_Purchase,Ratings,Date_Of_Review,Response,topic
0,0,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,zenheng,"delivery was quite fast, and item was on disco...",NaN,4.0,2/6/2020 23:58,no,delivery
1,1,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,glennoeestaquio,received item sealed properly.\r\r\r\r\nquick ...,NaN,5.0,2/6/2020 12:44,no,delivery
2,2,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,shy2206,yet to try but delivery was prompt and product...,NaN,5.0,2/6/2020 10:21,no,delivery
3,3,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,lindamiyalin,"well wrapped, in good condition, super fast ...",NaN,5.0,1/30/2020 23:01,no,delivery
4,4,Shopee,L'Oreal Paris,Serum & Treatment,L'Oreal Paris Revitalift Crystal Micro-Essence...,25.9,kyc385,"well packed, will buy again",NaN,5.0,2/1/2020 11:40,no,delivery


### Transformation and Initalise of data

In [4]:
# make the old csv data into a list

row_id_list = data['Unnamed:_0'].tolist()
platform_list = data['Platform'].tolist()
brand_list = data['Brand'].tolist()
category_list = data['Category'].tolist()
product_name_list = data['Product_Name'].tolist()
price_list = data['Price'].tolist()
reviewer_list = data['Reviewer'].tolist()
review_list = data['Review'].tolist()
product_variation_list = data['Product_Purchase'].tolist()
rating_list = data['Ratings'].tolist()
date_review_list = data["Date_Of_Review"].tolist()
response_list = data["Response"].tolist()
# dom_topic_list = data['Dorminant_Topic'].tolist()
topic_list = data["topic"].tolist()

# initialising the new columns

id_csv=[]
platform_csv=[]
brand_csv = []
category_csv = []
product_name_csv = []
prices_csv = []
reviewer_csv =[]
review_splitted_csv = []
review_csv=[]
product_variation_csv = []
rating_csv = []
date_review_csv = []
response_csv = []
# dom_topic_csv =[]
topic_csv=[]

# this is the length of the csv old data
print ("The length of the dataset is:", len(brand_list))

The length of the dataset is: 8987


### Transform the Date Format [Optional]

In [5]:
new_date =[]
for i in range(len(brand_list)):
    
    date_review = date_review_list[i]
    
    if platform_list[i] == 'Lazada':
        if 'days ago' in date_review or 'day ago' in date_review :
            day = date_review.replace('days ago', '').replace('day ago', '')
            date_review = datetime.today() - timedelta(days=int(day))
            date_review = date_review.strftime("%d-%b-%y")
#             print("trans",date_review)

        elif 'weeks ago' in date_review or 'week ago' in date_review:
            day = date_review.replace('weeks ago', '').replace('week ago', '')
            day = int(day)*7
            date_review = datetime.today() - timedelta(days=day)
            date_review = date_review.strftime("%d-%b-%y")
#             print("trans",date_review)

        elif 'hours ago' in date_review or 'hour ago' in date_review:
            hours = date_review.replace('hours ago', '').replace('hour ago', '')
            date_review = datetime.today() - timedelta(hours=int(hours))
            date_review = date_review.strftime("%d-%b-%y")
    
#         print("trans",date_review)
        new_date.append(date_review)
    else:
#         print(platform_list[i])
        new_date.append(date_review)
        
data["Date_Of_Review"] = new_date

new_date_review_list = data["Date_Of_Review"].tolist()

In [6]:
#upload the appos
from word_dict import appos

print(appos)

{"aren't": 'are not', "can't": 'cannot', "couldn't": 'could not', "didn't": 'did not', "doesn't": 'does not', "don't": 'do not', "hadn't": 'had not', "hasn't": 'has not', "haven't": 'have not', "he'd": 'he would', "he'll": 'he will', "he's": 'he is', "i'd": 'I had', "i'll": 'I will', "i'm": 'I am', "isn't": 'is not', "it's": 'it is', "it'll": 'it will', "i've": 'I have', "let's": 'let us', "l'oreal": 'loreal', "mightn't": 'might not', "mustn't": 'must not', "shan't": 'shall not', "she'd": 'she would', "she'll": 'she will', "she's": 'she is', "shouldn't": 'should not', "that's": 'that is', "there's": 'there is', "they'd": 'they would', "they'll": 'they will', "they're": 'they are', "they've": 'they have', "we'd": 'we would', "we're": 'we are', "weren't": 'were not', "we've": 'we have', "what'll": 'what will', "what're": 'what are', "what's": 'what is', "what've": 'what have', "where's": 'where is', "who'd": 'who would', "who'll": 'who will', "who're": 'who are', "who's": 'who is', "who'

### Spilt the Sentence 

In [7]:
import itertools
from spellchecker import SpellChecker
from nltk.tokenize import word_tokenize

spell = SpellChecker()

for i in range(len(brand_list)):
    # current_review is a string
    current_review = review_list[i]
    #print(current_review)
    current_review = current_review.strip().replace("’","'")
    

    #spilt the sentence
    splitted_sentence= re.split(r'[.!]', current_review) 
    print(splitted_sentence)
    
    for j in range(len(splitted_sentence)):
        
        splitted_sentence[j].strip()
        if len(splitted_sentence[j]) > 0 :
            
            processed_review = splitted_sentence[j].strip()
            
            # Negation handling
            processed_review = processed_review.split()
            processed_review =[appos[w] if w in appos else w for w in processed_review]
            processed_review = " ".join(processed_review) 
            
            processed_review = processed_review.strip()
            # Remove all the special characters
            processed_review = re.sub(r'\W', ' ', processed_review)
            # Removing prefixed 'b'
            processed_review = re.sub(r'^b\s+', '', processed_review)
            # remove all single characters contains a white space character
            processed_review= re.sub(r'^[a-zA-Z]$', ' ', processed_review)
            # Substituting multiple spaces with single space
            processed_review = re.sub(r'\s+', ' ', processed_review, flags=re.I)
          
            
            processed = processed_review.strip()
            #print(processed)
            
            # final check if the processed_review is empty
            final = []
            if processed != "":
                tokenized = word_tokenize(processed)
                #removed additional letters eg "gooooooodddddd" to "good"
                for w in tokenized:
                    processed_2 = reduce_lengthening(w)
                    processed_2 = spell.correction(processed_2)
                    final.append(processed_2)
                final =' '.join(final)
                #print(final)
                # append into a dictionary
                platform_csv.append(platform_list[i])
                id_csv.append(row_id_list[i])
                brand_csv.append(brand_list[i])
                category_csv.append(category_list[i])
                product_name_csv.append(product_name_list[i])
                prices_csv.append(price_list[i])
                reviewer_csv.append(reviewer_list[i])
                review_csv.append(current_review)
                review_splitted_csv.append(final)
                product_variation_csv.append(product_variation_list[i])
                rating_csv.append(rating_list[i])
                date_review_csv.append(new_date_review_list[i])
                response_csv.append(response_list[i])
                topic_csv.append(topic_list[i])


["delivery was quite fast, and item was on discount too so i couldn't complain", ' so far liking this product', ' will continue to use it :)']
['received item sealed properly', '\r\r\r\r\nquick delivery too', '\r\r\r\r\nitem not tested yet as for gifting purpose', '']
['yet to try but delivery was prompt and product has long expiry', '']
['well wrapped,  in good condition,  super fast delivery,  first time buy,  yet to try,  thanks seller  excellent service']
['well packed, will buy again']
['well wrap in good condition']
['came in good condition', ' will start using soon', ' thank u', '']
['soothing feel']
['super fast delivery, received item the following day', ' ordered during flash sale with special item coupon + coins', ' item is manufactured in jun 2019', ' good deal😃']
['effective product', ' fast delivery', ' received 1 day after order', '']
['received the products in a good condition and with fast 💨 delivery 🚚 service:)']
["it's definitely cheaper than retail with the coupon",

['items received in good condition thank you']
['product came quickly and incredibly well-packaged with a free gift :) thank you', '']
['good product quality, fast delivery']
['fast delivery, received with 2weeks', '', ' really worth the price, compare to shop is really much more cheaper and cone with the complexion', '', ' both manufacturers date are this year', '', ' thxqqq~~']
['received the items in good condition\r\r\r\r\nexpiry in 2022 \r\r\r\r\nworth buying', '']
['fast delivery and item was well received second time purchase and it came with uv sunblock', ' love to see more promo good buy']
['super fast', ' thanks']
['super love this product ', '', " so lucky to be able to purchase it from shopee, i went to watson the other day and couldn't find this", ' also the price is so much cheaper here than retail', '']
["received the product together with the sunscreen pretty quickly, can't wait to try them out", '', '']
['excited to try this', ' :-) came nicely packaged & delivered by 

['ordered on 30th jan, shipped and rcved on 31st jan via urbanfox delivery', ' i was able to use my remaining shoppee coins and the  loreal discount codes to offset the entire amt which made very worth it', ' have nvr use this 2 mask clay before so hopefully my face wont get negative effect', '']
['tried all 3 personally like the anti blemish most due to the watery formula easy to wash off', ' but all 3 masks after washed off, does make me skin soft & bouncy', '']
['price during offer is great for a clay mask (got it for $12) i only applied to my cheeks & nose area, after washing off, my skin feels slightly tight and dry upon touching after cleaning my cleansed face with tissue', ' and i have oily- combi skin', " i guess it's value for $ only", ' 6', '5/10']
['received fast after the cny break', ' good product', ' authentic', ' very cheap during sale at $10']
['buying again because this product works well', ' 10/10, will buy again', '']
['great purchase', '', ' used this mask before so

['ordered on 9sept, received on 16sept', ' yet to try', ' mfg date apr 2019, 50g', ' the white perfect milky foam is only 50ml, small, exp date may 2022']
['arrived really fast', ' got it in just two to three days', ' tried the face wash and mask, and so far so good', ' i have sensitive skin so usually my skin disagrees with a lot of products and i get breakouts, but not with this product', '']
['delivery was very quick', ' received in good condition', " product smells good, can't wait to try it out", ' thanks', ' :)']
['very fast delivery', ' order received within 2 days', " can't wait to try it", '']
['items well recieved in a bubble wrapped envelope', ' packaging of items is sealed', ' good purchase', '']
['feels nice, dries very quickly', ' the face wash also makes skin extremely soft and poreless']
['second time purchasing', ' this is really good at lightning my blemish on my check that i have had for the past 3 years', ' use 3 times a week with fancl renewal pack followed by anti

['great packaging, fast delivery', ' will use it tonight', ' thank you', '']
['nice blue colour, didnt mind the smell', '', ' there was a warm sensation when applied', '', " as for the effects, it's too early to tell if it makes a difference to my skin"]
['fast delivery', ' item received and well packed']
['item was recieved in good condition', ' delivery process was very fast too', " haven't use the products yet", '']
['fast delivery', ' the item was well wrapped and in good condition', '']
['got the items during the time sale', ' very worth and delivery was fast', '']
['received, but delivery was took pretty long', ' manufactured in 06/18']
['arrived well packed', " hasn't try yet so can't comment"]
['received in good condition, nice packing and delivery lead time very fast', '']
["wrapped nicely & can't wait to try this out", '']
['well received within a week', ' mfg in aug 2018', " haven't tried this product before", '']
['no comments/review is an image']
['delivery was very fast a

...

["i'm always a fan of lip products but can't find one that is really moisturising and can last me the whole day without touching up", " all the products i tried claims that it can last the whole day but to my disappointment it didn't work and furthermore made my lips dry", " i'm thankful and lucky to be able to try this amazing product and i can say it last me more than 16hr", " i think it's super worth it because u can look pretty the whole day without constant touching up"]
['it is abit dry', ' but it is very lasting', ' after meals most of my lipstick still stay', "  however i have dry lips so it's cracks abit"]
['glides on smoothly', '  not as dry as majority of matte lipsticks', "  doesn't rub off easily even with greasy food", '  some cracking and wrinkling after eating and some pale spots but still looks almost flawless', '']
['went out for the night and had steamboat for dinner', " was amazed at how i didn't have to reapply the lipstick after dinner because it was so long-lasti

['the superstay matte ink from maybelline is perfect for everyday wear', " it smells good, super pigmented and with the perfect contoured applicator, it's super easy & fast to apply on for panic mommies like me", ' just so u know, i tested it for a whole day of household chores', ' it lasted', ' i drank with it, eat with it', ' had to remove it with a makeup remover frankly', ' didnt stain my teacups too', ' honestly, i love it', ' its not drying unlike my other matte lippies', ' ok im buying other colours definitely', '', '']
['the lip tint is a bit sticky when apply but the longevity of the lip tint  stays on it when eat and drink']
['i love how the pigment is strong and lasts throughout the day, literally', '  however, the wand seems weird and its a bit hard to apply', ' also, it has a sticky senstation through', ' not forgetting to mention, it was so difficult to remove it', '', '']
['really amazed how non drying this liquid lipstick is', '     blot the excess and it will set non s

["i had a heavy lunch and some desserts for tea break while wearing this and i was pleasantly surprised that i didn't have to reapply throughout the day", '', '']
["lipstick fades away fast from my lips and i've tried a few brands", " if it doesn't disappear fast, i think leaves my lips dry", '     this lipstick is exactly what my lips needs', "     colour range is awesome - though i'd love for them to have more", '     application is smooth', ' one dip is more than enough', ' trust me', ' all you need i say to apply thin - or at least enough to cover your lips', '     it stays more than 16hours', " i know, i'm not supposed to, but i accidentally slept with it", ' it stayed until the next day', " that's about 24hrs", "    doesn't smudge too unless it's exposed to liquids for too long", ' ie: in the shower for 10 minutes with constant water running down my face', " with that said, there's no transfers when eating or drinking", '     oh', " it also doesn't leave my lips dry", '     smell

['okay so this is an interesting formular for a matte lippies', ' i have tons of reason to love it', "     - it's quick drying and transferproof", "   - it's sticky before it's dries but its normal for a matte lippies", '   - it has a velvet felt when u first applied but has a silky felt after', " ( i even tried using lip balm over it and it's okay", ")   - not only it's pigmented and has vibrant colour, the one i received (creator) is a purple but has pink undertone and even after a greasy dinner i had it fade off and left a tint", " which is great  - the applicator of it's great for matte lippies as i can outline my lips with it", '     the only concern i have was, removing it', " but there's a specific “eraser” for it", ' or else it can last for almost a day', "   i'll definitely use it again and buy the other colours too", '']
['no comments/review is an image']
['no comments/review is an image']
['no comments/review is an image']
['lipstick is super pigmented with no doubt', ' not 

['ordered on 24apr and received on 30apr in the mailbox', ' it can cover my dark spots', ' it doesnt crease in my eye area', '']
['exactly what i ordered', ' thanks', '']
['fast delivery', ' yet to try it', '']
['great product have repurchased more than 10 times']
['very worth to buy during 11', '11 sales', ' first time use n fall in love with it', ' it can blend well with my dry skin', ' will buy again ', '']
['came within a week']
['a very good buy', ' the manufacturing date is jan 2019', '']
['item received in good condition', ' speedy delivery as well', '']
['item delivered in good order', ' good as advertised', '']
["haven't used it yet but when i swatch it, the colour is very pink toned and doesn't match me", " also didn't receive any makeup removers at all when the picture shows that you'll receive 3 free with the purchase of this", ' misleading info', '']
['bought this during the 9', '9 flash deal and promised to have come with the makeup remover', ' but totally none', ' only r

In [8]:
#Store into a new data frame
splitted_data = {'ID':id_csv,'Platform':platform_csv ,'Brand':brand_csv, 'Category': category_csv, 'Product Name ': product_name_csv, 'Price':prices_csv ,'Reviewer':reviewer_csv,'Review':review_csv, 'Review_splitted':review_splitted_csv, 'Product Purchase':product_variation_csv,'Ratings':rating_csv,'Date Of Review':date_review_csv,'Response': response_csv , 'Topic':topic_csv}
splitted_df = pd.DataFrame.from_dict(splitted_data)
splitted_df['Date Of Review'] = pd.to_datetime(splitted_df['Date Of Review'])
splitted_df.head()
splitted_df.info()

splitted_df.to_csv('AllData_Process_Sent.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17119 entries, 0 to 17118
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ID                17119 non-null  int64         
 1   Platform          17119 non-null  object        
 2   Brand             17119 non-null  object        
 3   Category          17119 non-null  object        
 4   Product Name      17119 non-null  object        
 5   Price             17119 non-null  float64       
 6   Reviewer          17097 non-null  object        
 7   Review            17119 non-null  object        
 8   Review_splitted   17119 non-null  object        
 9   Product Purchase  14421 non-null  object        
 10  Ratings           17097 non-null  float64       
 11  Date Of Review    17097 non-null  datetime64[ns]
 12  Response          17097 non-null  object        
 13  Topic             17119 non-null  object        
dtypes: datetime64[ns](1), 